In [3]:
import torch
from torchvision import models
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import imageio

In [ ]:
# 이미지 데이터를 다루는 파이토치 모듈은 텐서가 채널x높이x너비 순으로 배치되어야 한다.

img_arr = imageio.imread('/Users/parkmiryeong/Desktop/연구인턴/무제 폴더/machine-learning/파이토치딥러닝마스터/1583915988281.jpg')
img_arr.shape

/var/folders/8l/9jsw0prj36ngswky608_l6rc0000gn/T/ipykernel_58325/991843658.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('/Users/parkmiryeong/Desktop/연구인턴/무제 폴더/machine-learning/파이토치딥러닝마스터/1583915988281.jpg')


(600, 800, 3)

In [ ]:
# 레이아웃 변경하기 permute()

img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1) # 주의) 텐서 복사본을 만들지 않음

# out은 img 저장 공간과 동일ㄹ하며, 단순히 텐서 레벨에서 크기와 스트라이드 정보만 변경되었다. 
# img 에서 픽셀을 변경하는 경우, out에 있는 픽셀도 변경된다.

out.shape

torch.Size([3, 600, 800])

In [ ]:
# 배치x채널x높이x너비 텐서로 저장하기

batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
batch.shape

torch.Size([3, 3, 256, 256])

In [ ]:
# 데이터 정규화

# 신경망은 일반적으로 부동소수점 텐서를 입력으로 사용한다.
# 0~1 사이거나 -1~1 사이일 때 훈련 성능이 가장 좋다.

batch = batch.float()
batch /= 255.0


In [ ]:
# 입력 데이터의 평균과 표준 편차를 구해서 평균이 0이고 각 채널값이 표준 편차를 넘지 않게 만들수도 있다.

n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

# 와인 데이터셋

In [5]:
import csv

wine_path = '/home/piai/문서/machinelearning/파이토치딥러닝마스터/data/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype = np.float32, delimiter=';',
                         skiprows=1) # 첫 행 생략

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [6]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [7]:
# 넘파이 배열을 파이토치 텐서로 변환

wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype  # 부동소수점 tensor를 만들었다.

(torch.Size([4898, 12]), torch.float32)

### 점수 표현하기


In [9]:
data = wineq[:, :-1] # 모든 행, 마지막제외 모든 열
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [10]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [11]:
# target 텐서 전치하기

target = wineq[:, -1].long() # 정수 벡터로 처리
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [28]:
# unsqueeze : 싱글톤 차원 추가
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed.shape, target_unsqueezed

(torch.Size([4898, 1]),
 tensor([[6],
         [6],
         [6],
         ...,
         [6],
         [7],
         [6]]))

In [29]:
print(target.shape)
print(target_unsqueezed.shape)

torch.Size([4898])
torch.Size([4898, 1])


In [25]:
# 원핫 인코딩 scatter_
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [26]:
target_onehot.scatter_(1, target.unsqueeze(1), 1.0) # 각 행에서 대상이 되는 레이블의 인덱스를 취한 다음(와인의 품질점수), 열 인덱스 값으로 사용해서 해당 항목의 값을 1.0 으로 지정한다
# 파라미터 설명
# 1번째: 몇 번째 차원에 값을 할당할건지
# 2번째: 원핫으로 인코딩할 요소를 가리키는 인덱스가 들어있는 텐서
# 3번째: 원핫 인코딩할 원소가 들어있는 텐서 혹은 단일 스칼라(1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


여기서 각 인자의 역할은 다음과 같습니다:

* 1: 차원을 나타냅니다. 1번째 차원에 값을 할당하겠다는 의미입니다. (4898)

* target_onehot: One-Hot 인코딩 결과를 저장할 텐서입니다.

* 1: 할당할 값입니다. 여기서는 1.0을 할당하고 있습니다.


이는 주로 분류 문제에서 사용되며, target 텐서에 있는 정수값이 해당하는 클래스의 인덱스를 나타내기 때문에 One-Hot 인코딩에서 해당 위치에 1을 할당하고 나머지 위치에는 0을 할당합니다.
즉, 이 코드는 target_onehot 텐서의 각 행에서 target에 해당하는 위치에 1.0을 할당하여 One-Hot 인코딩을 수행합니다.

# 시계열 데이터 다루기

In [30]:
bikes_numpy = np.loadtxt(
    '/home/piai/문서/machinelearning/파이토치딥러닝마스터/data/hour-fixed.csv',
    dtype= np.float32,
    delimiter=',',
    skiprows=1,
    converters={1: lambda x: float(x[8:10])} # 첫번째 열의 일자 문자열을 숫자로 변환
)

In [31]:
bikes_numpy

array([[1.0000e+00, 1.0000e+00, 1.0000e+00, ..., 3.0000e+00, 1.3000e+01,
        1.6000e+01],
       [2.0000e+00, 1.0000e+00, 1.0000e+00, ..., 8.0000e+00, 3.2000e+01,
        4.0000e+01],
       [3.0000e+00, 1.0000e+00, 1.0000e+00, ..., 5.0000e+00, 2.7000e+01,
        3.2000e+01],
       ...,
       [1.7377e+04, 3.1000e+01, 1.0000e+00, ..., 7.0000e+00, 8.3000e+01,
        9.0000e+01],
       [1.7378e+04, 3.1000e+01, 1.0000e+00, ..., 1.3000e+01, 4.8000e+01,
        6.1000e+01],
       [1.7379e+04, 3.1000e+01, 1.0000e+00, ..., 1.2000e+01, 3.7000e+01,
        4.9000e+01]], dtype=float32)

In [33]:
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

* 데이터를 시간 별로 보고싶다 -> N차원은 시간 축이 된다.

* 2년치 데이터셋을 쪼개서 일 단위처럼 넓은 관찰주기로 나누고 싶어 -> 길이가 L인 C개의 시퀀스를 가지는 N개의 컬렉션
* 시계열 데이터셋이 3차원 센서가 되어 NxCxL 로 만들어진다.
* 17520 = Nx17x24

In [34]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [40]:
# 데이터를 일자, 시간, 17개 열의 세 개 축으로 만들기

daliy_bikes = bikes.view(-1, 24, bikes.shape[1]) # view: 저장 공간을 바꾸지 않고 차원 수나 스트라이드 정보를 바꾼 새 텐서를 넘겨준다
# 새로 받을 텐서가 가질 차원 정보: (-1, 24, 17)
# -1: 남은 차원과 거기에 들어있는 요소를 다 합친것 (플레이스 홀더)

daliy_bikes.shape, daliy_bikes.stride() # 두번째 차원인 시간을 한 칸 이동하려면 17칸 이동, 첫번째 차원인 일자를 한 칸 이동하려면 17x24=408칸 이동

(torch.Size([730, 24, 17]), (408, 17, 1))

### 핵심 요약

* 신경망에서는 데이터가 다차원 숫자 텐서로 표현되어야 하며 대부분 32비트 부동소수점을 사용한다.
* 통상 파이토치에서 데이터는 컨볼루션이나 순환 등의 모델 아키텍처에 맞는 특정 차원으로 만들어져야 한다. 데이터는 파이토치 텐서 API를 사용하면 효율적으로 변환할 수 있다.
* 감사하게도 파이토치 라이브러리는 파이썬 표준 라이브러리 및 관련 생태계와 긴밀하게 상호작용하므로 일반적인 데이터 타입을 읽어들이거나 파이토치 텐서로 변환하기가 매우 편리하다.
* 이미지는 한 개 이상의 채널로 구성되어있다. 대부분은 디지털 사전의 rgb 채널에 해당한다.
* 많은 경우 이미지는 채널당 8비트를 가지며 12나 16비트도 특이한 경우는 아니다. 이런 비트 깊이는 모두 정밀도 손실 없이 32비트 부동소수점 수로 보관할 수 있다.
* 채널이 하나인 데이터는 채널 자체의 차원을 생략한 경우가 있다.
* 용적 데이터는 2차원 이미지 데이터와 유사하며 심도 같은 세 번째 차원이 존재하는 차이가 있다.
* 스프레드시트를 텐서로 변환하는 것은 매우 단순하다. 이 때 카테고리값이나 순서값이 들어있는 열은 이산 값으로 이루어진 열과는 다른 방식으로 처리해야 한다.
* 텍스트나 카테고리 데이터는 사전을 이용햔 원핫 표현으로 인코딩할 수 있다. 많은 경우 임베딩은 효율적이면서 좋은 표현 방법이다.